# CVaR ANN pipeline

## Evironment setup

Setup the general environment

In [1]:
import os, sys
from pathlib import Path

main_folder = str(Path.cwd().parent)
sys.path.append(main_folder)

import warnings
warnings.filterwarnings("ignore")

## Market simulator

Simulation of the variable annuity features and market environment

In [2]:
from src.utils import *
from src.features.market_simulator import *

config_file = load_config(os.path.join(main_folder,'cfgs','config_market_simulation.yml'))
market_model = market_simulator_class(config_file)
market_model.simulation()
market_model.delta_features()
market_model.delta_hedging()

-- Market simulation starts --
--Progress of market simulation: 100.00%
--Storing market features--

-- Hedging instruments array stored --

-- Gains and losses array stored --

-- Deltas input array stored --

-- Cash flow array stored --

-- Input array stored --

--Simulation of simulation features completed--
-- Delta hedging computation starts --
-- Progress of features computation: 
 Shap:8-Simulation:1000-Time:239
--Storing delta hedging features--
--Delta hedging features computation completed--
-- Positions computation starts --

--Positions computation completed and stored--


## CVaR ANN estimation 

CVaR agent training

**Remark 1**: This segment of the pipeline assumes that the market environment has already been simulated and trained, respectively.

**Remark 2**: The training of the CVaR net involves three steps:
- Training an ANN to estimate the variable annuity price,
- Estimating CVaR cash flows using Monte Carlo simulation to generate output data for the ANN,
- Training the CVaR net.


### Pricing agent

In [3]:
from src.utils import *
from src.models.pricing_agent_training import rl_agent

config_file = load_config(os.path.join(main_folder,'cfgs','config_pricing_agent.yml'))
config_file_agent = config_file["agent"]
config_file = load_config(os.path.join(main_folder,'cfgs','config_market_simulation.yml'))
config_file_simulation = config_file["simulation"]
_ = rl_agent(config_file_simulation,config_file_agent)

-------------------------------------------------------------
Pricing_agent_FFNN_dropout_50_pricing_va_assupmtion_240_
-------------------------------------------------------------
-----------------------Training start------------------------
Epoch 1, Time elapsed: 0:00:00.185559  - Model saved
  Train - MSE: 138.15474, Valid: 9.58581
Epoch 2, Time elapsed: 0:00:00.295119  - Model saved
  Train - MSE: 6.06847, Valid: 4.50145
Epoch 3, Time elapsed: 0:00:00.400785  - Model saved
  Train - MSE: 2.93782, Valid: 1.66104
Epoch 4, Time elapsed: 0:00:00.506477  - Model saved
  Train - MSE: 1.08782, Valid: 0.60792
Epoch 5, Time elapsed: 0:00:00.609865  - Model saved
  Train - MSE: 0.32476, Valid: 0.15597
Epoch 6, Time elapsed: 0:00:00.718765  - Model saved
  Train - MSE: 0.10281, Valid: 0.06556
Epoch 7, Time elapsed: 0:00:00.824289  - Model saved
  Train - MSE: 0.05004, Valid: 0.03613
Epoch 8, Time elapsed: 0:00:00.934412  - Model saved
  Train - MSE: 0.02902, Valid: 0.02211
Epoch 9, Time elaps

### CVaR estimation 

In [4]:
from src.features.cvar_estimation import cvar_estimation_

config_file_agent = load_config(os.path.join(main_folder,'cfgs','config_pricing_agent.yml'))
config_file_simulation = load_config(os.path.join(main_folder,'cfgs','config_market_simulation.yml'))
estimation_ = cvar_estimation_(config_file_simulation,config_file_agent)
_ = estimation_.estimation()

-- CVaR estimation starts --
-- Progress of CVaR estimation: 

INFO:tensorflow:Restoring parameters from Pricing_agent_FFNN_dropout_50_pricing_va_assupmtion_240_.ckpt


 Factor: -1 - Simulation:999 - Time:239
--Storing CVaR estimation--

-- CVaR estimation stored --


### CVaR ANN estimation

In [5]:
from src.models.cvar_agent_training import rl_agent

config_file = load_config(os.path.join(main_folder,'cfgs','config_cvar_agent.yml'))
config_file_agent = config_file["agent"]
config_file = load_config(os.path.join(main_folder,'cfgs','config_market_simulation.yml'))
config_file_simulation = config_file["simulation"]
_ = rl_agent(config_file_simulation,config_file_agent)

-------------------------------------------------------------
FFNN_dropout_50_cvar_net_assupmtion_240_
-------------------------------------------------------------
-----------------------Training start------------------------
Epoch 1, Time elapsed: 0:00:00.363220  - Model saved
  Train - MSE: 6.34087, Valid: 0.00006
Epoch 2, Time elapsed: 0:00:00.649269  - Model saved
  Train - MSE: 0.00007, Valid: 0.00006
Epoch 3, Time elapsed: 0:00:00.925747  - Model saved
  Train - MSE: 0.00006, Valid: 0.00005
Epoch 4, Time elapsed: 0:00:01.206280  - Model saved
  Train - MSE: 0.00005, Valid: 0.00004
Epoch 5, Time elapsed: 0:00:01.485974  - Model saved
  Train - MSE: 0.00005, Valid: 0.00004
Epoch 6, Time elapsed: 0:00:01.764869  - Model saved
  Train - MSE: 0.00004, Valid: 0.00003
Epoch 7, Time elapsed: 0:00:02.046980  - Model saved
  Train - MSE: 0.00004, Valid: 0.00003
Epoch 8, Time elapsed: 0:00:02.312338  - Model saved
  Train - MSE: 0.00004, Valid: 0.00003
Epoch 9, Time elapsed: 0:00:02.588479

INFO:tensorflow:Restoring parameters from FFNN_dropout_50_cvar_net_assupmtion_240_.ckpt


-------------------------------------------------------------
assupmtion_240_
-------------------------------------------------------------
---Inference start---
--- Storing CVaR agent parameters in ../models/.. ---
--- CVaR net parameters stored ---
